# Ward2Vec
- 문장 내부의 단어를 벡터로 변환하는 도구
- 단어의 연결을 기반으로 단어의 연관성을 벡터로 생성
- RNN이랑은 틀린방식
- 이를 활용하면 단어간의 유사도를 파악하여 단어의 의미를 파악 가능하다.
- 특히 한국어에서 많이 사용
- 단어의 의미를 선형으로 계산 가능.   
예) 아빠 - 남자 + 여자 = 엄마 , 왕자 - 남성 + 여성 = 공주 
- 여기에서 로봇채팅이 나온거임.

### Gensim 설치
- Word2Vec를 구현하는 도구중 실행속도가 빠름

### 허균의 홍길동전

In [1]:
# !pip install konlpy

In [2]:
# !pip install gensim

In [2]:
from konlpy.tag import Okt
from gensim.models import word2vec

In [5]:
file = open("../Data/홍길동전.txt", 'r')
text = str(file.readlines())
# text

In [12]:
# 텍스를 한 줄씩 처리하기
twitter = Okt()
results = []
lines = text.split("\n")
for line in lines:
    # 형태소 분석하기
    malist = twitter.pos(line,norm=True,stem=True) # 단어의 기본형 사용
    r = []
    for word in malist:
        # 어미, 조사, 구두점 등은 대상에서 제외
        if not word[1] in ["Josa","Eomi","Punctuation"]:
            if word[0] != "n":
                r.append(word[0]) # word[0] : data
    rl = (" ".join(r).strip())
    results.append(rl)
    # print(rl)

In [7]:
# 파일로 출력하기 
wakati_file = "../Data/홍길동전.wakati"
with open(wakati_file, 'w', encoding = 'utf-8') as fp:
    fp.write("\n".join(results))

In [8]:
# word2vec 모델 만들기 
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, window = 10, hs = 1, min_count=2, sg = 1) 
# hs = 1 (숫자, 특수문자 제외)
# sg = 1 (skip gram, 문법 체크 제외)
model.save("../Data/홍길동전.model")
print("OK")

OK


In [9]:
# 모델 사용하기 
model = word2vec.Word2Vec.load("../Data/홍길동전.model")

In [10]:
# 가장 의미 비슷한 단어 찾기 
model.wv.most_similar(positive = ['백성'])

[('재물', 0.949138343334198),
 ('놈', 0.940064013004303),
 ('도적', 0.9196517467498779),
 ('수령', 0.9179301857948303),
 ('앗', 0.9166228771209717),
 ('곡식', 0.9151845574378967),
 ('불쌍하다', 0.907372772693634),
 ('취하', 0.9025781154632568),
 ('넉넉하다', 0.8986300826072693),
 ('불의', 0.8985934257507324)]

In [11]:
model.wv.most_similar(negative=['백성'])

[('부인', -0.27256086468696594),
 ('대감', -0.33682015538215637),
 ('말', -0.4113878607749939),
 ('들다', -0.48818403482437134),
 ('대하', -0.4961871802806854),
 ('내', -0.5140997767448425),
 ('새', -0.5150015354156494),
 ('들어가다', -0.519146740436554),
 ('무슨', -0.5414847731590271),
 ('알', -0.5451776385307312)]

---
 ## 위키피디아 한국어 버전을 사용하기


위키디피아 데이터 다운로드 : https://dumps.wikimedia.org/kowiki/latest/ 에서 kowiki-latest-pages-articles.xml.bz2를 다운  
bzip2 -d kowiki-latest-pages-articles.xml.bz  
sudo gem install wp2txt  
wp2txt --input-file ./kowiki-latest-pages-articles.xml (약 20분 소요)  
cat kowiki-latest-pages-articles-* > wiki.txt  

In [ ]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from gensim.models import word2vec

# 파일 열기
readFp = codecs.open("../../wiki/wiki.txt", "r", encoding="utf-8")
wakati_file = "../../wiki/wiki.wakati"
writeFp = open(wakati_file, "w", encoding="utf-8")

# 형태소 분석 -
twitter = Okt()
i = 0

# 텍스트를 한 줄씩 처리하기
while True:
    line = readFp.readline()
    if not line: break
    if i % 20000 == 0:
        print("current - " + str(i))
    i += 1
    # 형태소 분석
    malist = twitter.pos(line, norm=True, stem=True)
    # 필요한 어구만 대상으로 하기
    r = []
    for word in malist:
        # 어미/조사/구두점 등은 대상에서 제외 
        if not word[1] in ["Josa", "Eomi", "Punctuation"]:
            writeFp.write(word[0] + " ")
writeFp.close()

In [ ]:
from gensim.models import word2vec
data = word2vec.Text8Corpus("../../wiki/wiki.wakati")
model = word2vec.Word2Vec(data)
model.save("../../wiki/wiki.model")
print("ok")

---
위 과정은 오래걸림.

In [ ]:
# 사용하기 
from gensim.models import word2vec
model2 = word2vec.Word2Vec.load("../Data/wiki/wiki.model")

In [14]:
model2.wv.most_similar(positive=['Python',"파이썬"])

[('Perl', 0.9049678444862366),
 ('Java', 0.8980178833007812),
 ('JavaScript', 0.885010838508606),
 ('Tcl', 0.8817355036735535),
 ('Lisp', 0.862151563167572),
 ('자바스크립트', 0.8619224429130554),
 ('js', 0.8576772809028625),
 ('Qt', 0.8527214527130127),
 ('JSP', 0.8512636423110962),
 ('OpenGL', 0.8508063554763794)]

In [15]:
model2.wv.most_similar(positive=['아빠',"여성"], negative=['남성'])[0]

('엄마', 0.8462279438972473)

In [16]:
model2.wv.most_similar(positive=['왕자',"여성"], negative=['남성'])[0:5]

[('공주', 0.6534191966056824),
 ('왕', 0.6481793522834778),
 ('여왕', 0.6432251334190369),
 ('왕녀', 0.6313380599021912),
 ('왕비', 0.6241751313209534)]

In [17]:
model2.wv.most_similar(positive=['강남',"맛집"])

[('커피숍', 0.657056987285614),
 ('랜선', 0.642453134059906),
 ('청담동', 0.6393517851829529),
 ('횟집', 0.636467456817627),
 ('홍대', 0.6354712247848511),
 ('인사동', 0.6221076846122742),
 ('먹방', 0.6165238618850708),
 ('주점', 0.6155254244804382),
 ('장보기', 0.6106765270233154),
 ('한식당', 0.6068207621574402)]